# Homework 4
#### Yufeng Yuan
#### yy208@duke.edu

In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import subprocess
import skimage
import skimage.io as imio
import skimage.transform as trans
import skimage.filters as filters
from skimage.viewer import ImageViewer
import glob
import re
import os
from moviepy.editor import *
from time import gmtime, strftime, sleep
from scipy import interpolate as interp
%matplotlib inline
mpl.rcParams['figure.figsize'] = (16.0, 8.0)

/home/arthur/Softwares/anaconda3/lib/python3.6/site-packages/skimage/viewer/utils/core.py:10: UserWarning: Recommended matplotlib backend is `Agg` for full skimage.viewer functionality.
  warn("Recommended matplotlib backend is `Agg` for full "


In [85]:
def move_focus(sid=1, step=0):
    cmD='rsh ubuntu@10.0.1.7 "sudo python2 ~/Desktop/moveFocusMotor.py '+str(sid+1)+' '+str(step)+'"'
    pop=subprocess.run(cmD, stdout=subprocess.PIPE, shell=True)
    #return pop.stdout

def take_photos(sid=1, step=0, iters=1, start=0):
    subprocess.run('rm dbGrab*', \
               stdout=subprocess.PIPE, shell=True)
    subprocess.run('rsh ubuntu@10.0.1.7 "rm dbGrab*"', \
               stdout=subprocess.PIPE, shell=True)
    cmD='rsh ubuntu@10.0.1.7 "sudo python2 ~/Desktop/moveFocusMotor.py '+str(sid+1)+' '+str(start)+'"'
    subprocess.run(cmD, stdout=subprocess.PIPE, shell=True)
    for i in range(iters):
        subprocess.run('rsh ubuntu@10.0.1.7 "nvgstcapture-1.0 -m 1 --sensor-id '+str(sid)+' -A --capture-auto 1 \
                --file-name dbGrab_'+str(i*step)+' --image-res 8"', stdout=subprocess.PIPE, shell=True)
        cmD='rsh ubuntu@10.0.1.7 "sudo python2 ~/Desktop/moveFocusMotor.py '+str(sid+1)+' '+str(step)+'"'
        subprocess.run(cmD, stdout=subprocess.PIPE, shell=True)
    subprocess.run('rcp ubuntu@10.0.1.7:dbGrab* .', \
                stdout=subprocess.PIPE, shell=True)
    pop=subprocess.run('ls',stdout=subprocess.PIPE, shell=True)
    rePop=pop.stdout
    rePop=rePop.decode("utf-8")
    fileName = re.search(r'dbGrab(.*).jpg', rePop)
    fileName=fileName.group()
    pop=imio.imread(fileName)
    return pop

def read_eval(x=0, y=0, h=2160, w=3840):
    names = os.listdir()
    pics = []
    scores = []
    for name in sorted(names):
        if name[:6] == 'dbGrab':
            pic = imio.imread(name)
            pics.append(pic)
            scores.append(np.sum(filters.sobel(pic[x:x+w, y:y+h,0])))
    ind = scores.index(max(scores))
    #imio.imshow(pics[ind])
    focus = 100 * ind
    return focus, pics[ind], scores

def auto_focus(sid=1, region=[0,0,2160,3840]):
    take_photos(sid, 100, 10)
    x, y, h, w = region
    focus, pics, scores = read_eval(x, y, h, w)
    start = -1500
    cmD='rsh ubuntu@10.0.1.7 "sudo python2 ~/Desktop/moveFocusMotor.py '+str(sid+1)+' '+str(start)+'"'
    subprocess.run(cmD, stdout=subprocess.PIPE, shell=True)
    cmD='rsh ubuntu@10.0.1.7 "sudo python2 ~/Desktop/moveFocusMotor.py '+str(sid+1)+' '+str(focus+100)+'"'
    subprocess.run(cmD, stdout=subprocess.PIPE, shell=True)
    pop = take_photos(sid, 0, 1, 0)
    imio.imshow(pop)
    return focus, pop, scores

def grabVideo(sid=1, imres=8, cameraNumber=7, fps=30):
    # I will use the file name vidGrab.... for the image grabbed from the camera. First step is to get rid of previously grabbed images
    cmdA='rsh ubuntu@10.0.1.'+str(cameraNumber)
    cmdB='rcp ubuntu@10.0.1.'+str(cameraNumber)
    subprocess.run('rm vidGrab*', \
                   stdout=subprocess.PIPE, shell=True)
    subprocess.run(cmdA+' "rm vidGrab*"', \
                   stdout=subprocess.PIPE, shell=True)
    # ok, now let's grab an video on the tegra
    cmD=cmdA+' "nvgstcapture-1.0 -m 4 --sensor-id '+str(sid)+' -A '\
            +'--capture-auto 1 --fpsRange="'+str(fps)+' '+str(fps)+'"'\
            +'--capture-time 30 --file-name vidGrab --image-res '\
            + str(imres)+'"'
    subprocess.run(cmD, stdout=subprocess.PIPE, shell=True)
    subprocess.run(cmdB+':vidGrab* .', \
                   stdout=subprocess.PIPE, shell=True)
    #ok, now let's find the file name of the image we just grabbed 
    # and transfered 
    pop=subprocess.run('ls',stdout=subprocess.PIPE, shell=True)
    rePop=pop.stdout
    rePop=rePop.decode("utf-8")
    fileName = re.search(r'vidGrab(.*)', rePop)
    fileName=fileName.group()
    pop=VideoFileClip(fileName)
    return pop
def gstVideo(sid=1, cameraNumber=7, captureMode=1, frames=1200, fileOut="test.mp4", frameRate=240):
    cmdA='rsh ubuntu@10.0.1.'+str(cameraNumber)
    cmdB='rcp ubuntu@10.0.1.'+str(cameraNumber)
    cmD=cmdA+" sudo python2 /home/ubuntu/bin/grabVidModes.py "+str(sid)+' '+str(frames)+' '+str(captureMode)
    a=subprocess.run(cmD, stdout=subprocess.PIPE, shell=True)
    subprocess.run(cmdB+':grabVid.h264 .', \
        stdout=subprocess.PIPE, shell=True)
    subprocess.run('ffmpeg -r '+ str(frameRate) +' -i grabVid.h264 -an -y -vcodec copy '+fileOut, \
        stdout=subprocess.PIPE, shell=True)
    return a.stdout

### Task 3

In [ ]:
gstVideo(sid=1,captureMode=4, fileOut="clock_240.mp4", frameRate=240)
gstVideo(sid=1,captureMode=2, fileOut="clock_120.mp4", frameRate=120)
gstVideo(sid=1,captureMode=1, fileOut="clock_60.mp4",  frameRate=60)

These videos are too large or the frame rate is too high to be played in Jupyter Notebook, you may play them with VLC in your local machine

### Task 4

In [ ]:
gstVideo(sid=1,captureMode=1, fileOut="clock_240_10.mp4",  frameRate=10)
gstVideo(sid=1,captureMode=1, fileOut="water_240_20.mp4",  frameRate=20)

The interesting effects that I noticed is as follows. For the video of clock, whose original frame rate is 240 fps, if I play it at 10 fps, the change of 0.01s can be observed. This is impossible for a video of 30 or 60 fps.
Similarly, for the water video of 240 fps, if I play it at 10 fps, the continuity of the movement of bubble are more obvious than those of 30 or 60 fps.

### Task 5

In [ ]:
gstVideo(sid=1,captureMode=4, fileOut="dark_240.mp4", frameRate=240)
gstVideo(sid=1,captureMode=2, fileOut="dark_120.mp4", frameRate=120)
gstVideo(sid=1,captureMode=1, fileOut="dark_60.mp4",  frameRate=60)

From those videos, it obvious that in dark environment, higher frame rate causes more noise. I think this can be explained by the theory mentioned in class. The photons in dark environment is already limited, higher frame rate would make photons for each pixel in every frame rate decrease. The result is that the effect of noise would be more apparent.  